#Goal 1 is to Check if missing Planning CT that are presnt in Nii but no in NiiProcessed


#Goal 2 is to check the sizes of the CTS , all should be 320, if they are bigger check why, maybe cropping them is a good idea?


In [2]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import os
import random
from skimage.measure import label,regionprops
from LookFiles import LookFilesNiiRaw
import pandas as pd

In [3]:
#root = "Z:/Projectline_modelling_lung_cancer/Nii_Processed/"
root_NiiProcessed = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CT_ITV_GTV_XBP_Nii_Processed/"
root_NiiRaw = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CT_ITV_GTV_XBP_Nii/"
root_2SaveCSVs = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/"

In [4]:
PxList_path  = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/PxSelectionforDCM2Nii.txt"
with open(PxList_path, 'r') as file:
    PxList = [line.rstrip() for line in file]
print(len(PxList))

861


In [5]:
def LookFiles_Snapshot(folder_path):
    planCT_list,planCTLung_list = ([],[])
    bp50_list, bp50Lung_list = ([],[])
    gtvP_list, gtvN_list = ([],[])
    itvP_list, itvN_list = ([],[])
    for root,dirs,files in os.walk(folder_path):
        for f in files:
            if "PlanCT_CTProcessed.nii" in f:
                planCT_list.append(os.path.join(folder_path,f))
            if "PlanCT_LungProcessed.nii" in f:
                planCTLung_list.append(os.path.join(folder_path,f))
            if "bp50_CTProcessed.nii" in f:
                bp50_list.append(os.path.join(folder_path,f))
            if "bp50_LungProcessed.nii" in f:
                bp50Lung_list.append(os.path.join(folder_path,f))
            if "GTVn_GTProcessed.nii" in f:
                gtvN_list.append(os.path.join(folder_path,f))
            if "GTVp_GTProcessed.nii" in f:
                gtvP_list.append(os.path.join(folder_path,f))
            if "ITVn_GTProcessed.nii" in f:
                itvN_list.append(os.path.join(folder_path,f))
            if "ITVp_GTProcessed.nii" in f:
                itvP_list.append(os.path.join(folder_path,f))
    if len(bp50_list)==0: 
        for root,dirs,files in os.walk(folder_path):
            for f in files:
                if "bp60_CTProcessed.nii" in f:
                    bp50_list.append(os.path.join(folder_path,f))
                if "bp60_LungProcessed.nii" in f:
                    bp50Lung_list.append(os.path.join(folder_path,f))
    if len(bp50_list)==0: 
        for root,dirs,files in os.walk(folder_path):
            for f in files:
                if "bp40_CTProcessed.nii" in f:
                    bp50_list.append(os.path.join(folder_path,f))
                if "bp40_LungProcessed.nii" in f:
                    bp50Lung_list.append(os.path.join(folder_path,f))
    #print("Len of Files",len(planCT_list),len(planCTLung_list),len(bp50_list),len(bp50Lung_list),len(gtv_list),len(itv_list))


    return planCT_list,planCTLung_list,bp50_list,bp50Lung_list,itvP_list,itvN_list,gtvP_list, gtvN_list
    

In [6]:
def Screenshot(ct_np,gtv_np,middle_x,middle_y,middle_z,contourBool=True,Px2Save=None,ImageType=None):
    minHU = -1024
    maxHU = 600
    savebool = True
    dpi_val=100
    #root2save= "Z:/Projectline_modelling_lung_cancer/JPEGs/Processed/V1/"
    root2save= "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/JPEGs/Nii_Processed/"
    if Px2Save is not None:
        pathToSave = os.path.join(root2save,Px2Save)
        if not(os.path.exists(pathToSave)):
            os.mkdir(pathToSave)
            print(middle_x,middle_y,middle_z,Px2Save,ct_np.shape,ImageType,gtv_np.shape)
            clipped_image = np.clip(ct_np, minHU, maxHU)
            #middle_x,middle_y,middle_z = getMiddleSlices(gtv_np)
            plt.figure(figsize=(15, 15), dpi=dpi_val)#dpi=300)
            plt.imshow(clipped_image[:,:,middle_z],cmap='gray'),plt.axis('off')#,plt.colorbar()
            plt.title("Z:"+str(middle_z))
            if contourBool:
                plt.contour(gtv_np[:,:,middle_z],linewidths=0.5,colors='r')
            if not(savebool): 
                plt.show()
            else:
                plt.savefig(pathToSave+"/"+ImageType+"_Axial.jpeg")
                plt.close()
                plt.clf()
            plt.figure(figsize=(15, 15), dpi=dpi_val)
            plt.imshow(np.rot90(clipped_image[:,middle_y,:]),cmap='gray'),plt.axis('off')#,plt.colorbar()
            plt.title("Y:"+str(middle_y))
            if contourBool:
                plt.contour(np.rot90(gtv_np[:,middle_y,:]),linewidths=0.5,colors='r')
            if not(savebool): 
                plt.show()
            else:
                plt.savefig(pathToSave+"/"+ImageType+"_Sagital.jpeg")
                plt.close()
                plt.clf()
            plt.figure(figsize=(15, 15), dpi=dpi_val)
            plt.imshow(np.rot90(clipped_image[middle_x,:,:]),cmap='gray'),plt.axis('off')#,plt.colorbar()
            plt.title("X:"+str(middle_x))
            if contourBool:
                plt.contour(np.rot90(gtv_np[middle_x,:,:]),linewidths=0.5,colors='r')
            if not(savebool): 
                plt.show()
            else:
                plt.savefig(pathToSave+"/"+ImageType+"_Coronal.jpeg")
                plt.close()
                plt.clf()
        else:
            print("Not Saving Px")

In [7]:
def CreateIndices(struct_np):
    IndicesX,IndicesY,IndicesZ = [],[],[]
    for i in range(struct_np.shape[-3]):
        if np.sum(struct_np[i,:,:]):
            IndicesX.append(i)
    for i in range(struct_np.shape[-2]):
        if np.sum(struct_np[:,i,:]):
            IndicesY.append(i)
    for i in range(struct_np.shape[-1]):
        if np.sum(struct_np[:,:,i]):
            IndicesZ.append(i) 

    return IndicesX,IndicesY,IndicesZ

In [8]:
def SelectIndexAndScreenshot(Px,ContourType,indices_x,indices_y,indices_z,ct_np,struct_np):
    if len(indices_x)>0 and len(indices_y)>0 and len(indices_z)>0: 
        randomc_x = random.choice(indices_x)
        randomc_y = random.choice(indices_y)
        randomc_z = random.choice(indices_z)
        Screenshot(ct_np,struct_np,randomc_x,randomc_y,randomc_z,True,Px,ContourType)
    else:
        print(Px,"Indices not >0",ContourType)

In [9]:
def MainDisplayTumor(Px,ct_path,structP_path,structN_path,structTypeString,screenshot_bool):
    missingMaxExp,missingGTV,shapesErrorGTV,emptyGTVp,emptyGTVn = False,False,False,False,False
    if len(ct_path) ==0:
        missingMaxExp = True
    if len(structP_path)==0 and len(structN_path)==0:
        missingGTV = True
    if len(ct_path)>0 and len(structP_path)>0 and len(structN_path)>0:
        gtvp_np  = nib.load(structP_path[0]).get_fdata()
        gtvn_np  = nib.load(structN_path[0]).get_fdata()
        bp50_np  = nib.load(ct_path[0]).get_fdata()
        if gtvp_np.shape !=bp50_np.shape or len(gtvp_np.shape)==1:
            shapesErrorGTV = True
        else:
            indicespX,indicespY,indicespZ =CreateIndices(gtvp_np)
            if len(indicespX)>0 and len(indicespY)>0 and len(indicespZ)>0:
                if screenshot_bool:
                    SelectIndexAndScreenshot(Px,structTypeString+"p",indicespX,indicespY,indicespZ,bp50_np,gtvp_np)
                    print(Px,"Created",structTypeString,"p")
            else:
                emptyGTVp = True
            indicesnX,indicesnY,indicesnZ =CreateIndices(gtvn_np)
            if len(indicespX)>0 and len(indicespY)>0 and len(indicespZ)>0:
                if screenshot_bool:
                    SelectIndexAndScreenshot(Px,structTypeString+"n",indicesnX,indicesnY,indicesnZ,bp50_np,gtvn_np)
                    print(Px,"Created",structTypeString+"n")
            else:
                emptyGTVn = True
    return missingMaxExp,missingGTV,shapesErrorGTV,emptyGTVp,emptyGTVn

In [10]:
def main(screenshot_bool,root_NiiProcessed,PxList):
    listStatusPx = []
    for Px in PxList:
        planCT_list,_,bp50_list,_,itvP_list,itvN_list,gtvP_list, gtvN_list = LookFiles_Snapshot(root_NiiProcessed+Px)

        missingPlanCT,missingITV,shapesErrorITV,emptyITVp,emptyITVn = MainDisplayTumor(Px,planCT_list,itvP_list,itvN_list,"ITV",screenshot_bool)
        missingMaxExp,missingGTV,shapesErrorGTV,emptyGTVp,emptyGTVn = MainDisplayTumor(Px,bp50_list,gtvP_list,gtvN_list,"GTV",screenshot_bool)
        
        listStatusPx.append([Px,missingPlanCT,missingMaxExp, missingITV,missingGTV,shapesErrorITV,shapesErrorGTV,emptyITVp,emptyITVn,emptyGTVp,emptyGTVn])
    return listStatusPx

listStatusPx = main(True,root_NiiProcessed,PxList)

123 114 223 4291861 (320, 320, 320) ITVp (320, 320, 320)
4291861 Created ITV p
4291861 Indices not >0 ITVn
4291861 Created ITVn
Not Saving Px
4291861 Created GTV p
4291861 Indices not >0 GTVn
4291861 Created GTVn
169 137 276 3327503 (320, 320, 320) ITVp (320, 320, 320)
3327503 Created ITV p
Not Saving Px
3327503 Created ITVn
76 99 207 4693524 (320, 320, 320) ITVp (320, 320, 320)
4693524 Created ITV p
4693524 Indices not >0 ITVn
4693524 Created ITVn
Not Saving Px
4693524 Created GTV p
4693524 Indices not >0 GTVn
4693524 Created GTVn
155 115 175 6184000 (320, 320, 320) ITVp (320, 320, 320)
6184000 Created ITV p
Not Saving Px
6184000 Created ITVn
Not Saving Px
6184000 Created GTV p
Not Saving Px
6184000 Created GTVn
181 191 178 6163057 (320, 320, 320) ITVp (320, 320, 320)
6163057 Created ITV p
Not Saving Px
6163057 Created ITVn
Not Saving Px
6163057 Created GTV p
6163057 Indices not >0 GTVn
6163057 Created GTVn
151 184 189 659641 (320, 320, 320) ITVp (320, 320, 320)
659641 Created ITV p
6

<Figure size 640x480 with 0 Axes>

In [11]:
#Save PxList status on df
columns = ["Px","missingPlanCT","missingMaxExp", "missingITV","missingGTV","shapesErrorITV","shapesErrorGTV","emptyITVp","emptyITVn","emptyGTVp","emptyGTVn"]
df = pd.DataFrame(listStatusPx, columns=columns)
df.to_csv(root_2SaveCSVs+'NiiProcessed_Status_V2.csv', index=False)